In [3]:
# %md
### ingest circuits.csv file

![Alt text](data_ingestion.png)


In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerField, SringType, DoubleType

In [ ]:
#StructType represent the rows and StructField represents the columns

circuits_schema = StructType(fields=[
    StructField("circuitId", IntegerType(), False),
    StructField("name", StringType(), True),
    StructField("location", StringType(), True),
    StructField("country", StringType(), True),
    StructField("lat", DoubleType(), True),
    StructField("lng", DoubleType(), True),
    StructField("alt", IntegerType(), True),
    StructField("url", StringType(), True),
])

READ DATA

In [ ]:
# step 1 - read the csv file using the spark dataframe reader

In [ ]:
# display all the mounts point and source
display(dbutils.fs.mounts())

In [ ]:
# list all the path for a specific mount
%fs
ls /mnt/formula1dl/raw

In [ ]:
circuits_df = spark.read.csv("")

# this will display the headers in the columns
circuits_df = spark.read \
.option("header", True) \
.schema(circuits_schema) \
.csv("")

In [ ]:
#this will show the first 20 rows
circuits_df_show()

#this is better to show nicer data
display(circuits_df)

CHECK DATA TYPE

In [ ]:
#use printSchema from df to check data type BECAREFUL DOUBLE CHECK IF IT IS ACCURATE
circuits_df.printSchema()

In [ ]:
#this is more accurate for checking datatype
# the min and max return will show if the datatype is right

circuits_df.describe().show()

Select only the required columns

In [ ]:
#there are different ways to do this

circuits_selected_df = circuits_df.select("circuitId", "circuitRef", "name", "location", "country", "lat", "lng", "alt")

In [ ]:
#rename the columns

circuits_selected_df = circuits_df.select(col("circuitId").alias("circuit_id"), col("circuitRef").alias("circuit_ref"), col("name"), col("location"), col("country"),
col("lat"), col("lng"), col("alt"))

rename the columns as required

In [ ]:
circuits_renamed_df = circuits_selected_df.withColumnRenamed("circuitId", "circuit_id") \
    .withColumnRenamed("circuitRef", "circuit_ref") \
    .withColumnRenamed("lat", "latitude") \
    .withColumnRenamed("lng", "longitude") \
    .withColumnRenamed("alt", "altitude")

Add ingestion date to the dataframe

In [ ]:
from pyspark.sql.functions import current_timestamp

In [ ]:
circuits_final_df = circuits_renamed_df.withColumn("ingestion_date", current_timestamp())

write data to datalake as parquet

In [ ]:
#this will save the data as a parquet file
#the overwrite will replace the data if it already exists..there other options like display error, ignore and append
#partiion helps break the files down into smaller files
circuits_final_df_write.mode("overwrite").paritionBy("location").parquet("/mnt/formula1dl/processed/circuits")

In [ ]:
#this to check the file is saved
%fs
ls /mnt/formula1dl/processed/circuits

In [ ]:
df = spark.read.parquet("/mnt/formula1dl/processed/circuits")
display(df)